In [14]:
import pandas as pd
import argparse
from nilearn.glm.second_level import SecondLevelModel
from nilearn import plotting
import os
from nilearn.glm.second_level import make_second_level_design_matrix

group = "gangnam_total"
variable = "HADS_depression"
smoothness = 6
mdmr_dir = "/home/changbae/fmri_project/C-PAC/CPAC/bcb_mdmr/"
nas_dir = "/mnt/NAS2-2/data/"
MDMR_output_dir = f"{nas_dir}/SAD_gangnam_MDMR/"
fmri_prep_dir = f"{nas_dir}/SAD_gangnam_resting_2/fMRIPrep_total"
seed_anal_dir = f"{nas_dir}/SAD_gangnam_seed_based_analysis/"

regressor_df = pd.read_csv(
    f"{mdmr_dir}/input/{group}_{variable}_regressor.csv"
)

subjects_label = regressor_df["Participant"].values
# 필요한 열만 선택하여 디자인 매트릭스 생성

extra_info_subjects = pd.DataFrame({
    "subject_label": subjects_label,
    "sex": regressor_df["SEX"],
    "age": regressor_df["AGE"],
    "yr_edu": regressor_df["YR_EDU"],
    "mean_framewise_displacement": regressor_df["Mean_Framewise_Displacement"]
})
design_matrix = make_second_level_design_matrix(
    subjects_label, extra_info_subjects
)


/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/nilearn/glm/first_level/design_matrix.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1134644582781456' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  design_matrix.loc[ridx, conf_name] = confounds_value
/home/changbae/anaconda3/envs/fmrienv/lib/python3.9/site-packages/nilearn/glm/first_level/design_matrix.py:503: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '16.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  design_matrix.loc[ridx, conf_name] = confounds_value


In [15]:
second_level_model = SecondLevelModel(n_jobs=-1)
z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]

second_level_model = second_level_model.fit(
    z_maps,
    design_matrix=design_matrix,
)

In [16]:
z_map = second_level_model.compute_contrast("intercept", output_type='z_score')

result_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/result/"
temp_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/temp/"
os.makedirs(result_dir, exist_ok=True)
from nilearn.glm import threshold_stats_img
z_map_filename = f"{result_dir}/unthresholded_z_map.nii.gz"
z_map.to_filename(z_map_filename)

thresholded_map2, threshold2 = threshold_stats_img(
    z_map, alpha=0.05, height_control="fdr"
)
print(f"The FDR=.05 threshold is {threshold2:.3g}")
thresholded_map2_filename = f"{result_dir}/FDR_correted_z_map.nii.gz"
thresholded_map2.to_filename(thresholded_map2_filename)

thresholded_map3, threshold3 = threshold_stats_img(
    z_map, alpha=0.05, height_control="bonferroni"
)
print(f"The p<.05 Bonferroni-corrected threshold is {threshold3:.3g}")
thresholded_map3_filename = f"{result_dir}/bonferroni_correted_z_map.nii.gz"
thresholded_map3.to_filename(thresholded_map3_filename)

The FDR=.05 threshold is 2.15
The p<.05 Bonferroni-corrected threshold is 4.71


In [17]:
design_matrix = regressor_df[['SEX', 'AGE', 'YR_EDU', variable, 'Mean_Framewise_Displacement']]

# 인터셉트 열 추가
design_matrix.insert(0, 'intercept', 1)

z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]

second_level_model = SecondLevelModel(n_jobs=-1)
second_level_model.fit(z_maps, design_matrix=design_matrix)

z_map = second_level_model.compute_contrast(args.variable, output_type='z_score')

result_dir = f"{seed_anal_dir}/{args.smoothness}mm/p_map/{args.group}/{args.variable}/result/"
temp_dir = f"{seed_anal_dir}/{args.smoothness}mm/p_map/{args.group}/{args.variable}/temp/"
os.makedirs(result_dir, exist_ok=True)

p_map_filename = f"{result_dir}/pvalue_map.nii.gz"
p_map.to_filename(p_map_filename)

NameError: name 'args' is not defined